In [368]:
import json
import os

import folium
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pygris
from shapely.geometry import Polygon, MultiPolygon, GeometryCollection
from shapely.ops import unary_union
from shapely.validation import make_valid


# so geopandas plots look nice
%matplotlib inline

In [369]:
state_in = 'WA'
state_FIPS = '53'
counties_in = [ "King", "Pierce", "Snohomish", "Kitsap", "Skagit",
             "Island", "Thurston", "Lewis", "Mason", "Whatcom", 
             "San Juan", "Clallam", "Jefferson", "Grays Harbor",
             "Pacific", "Wahkiakum", "Cowlitz", "Clark", "Skamania"]
# -------------
sld_gdb_path = r"C:/Users/Soheil99/OneDrive - UW/0 Research/UW Tacoma/my copy - Satellite Communities Project/Data/SmartLocationDatabase.gdb"
pop_ctr_path = r"C:/Users/Soheil99/OneDrive - UW/0 Research/UW Tacoma/my copy - Satellite Communities Project/Data/WSDOT_-_Population_Centers/WSDOT_-_Population_Centers.shp"
nces_WA_path= r"C:/Users/Soheil99/OneDrive - UW/0 Research/UW Tacoma/my copy - Satellite Communities Project/Data/edge_locale24_nces_WA"
POI_path = r"C:/Users/Soheil99/OneDrive - UW/0 Research/UW Tacoma/my copy - Satellite Communities Project/Data/POI Data/WA_Study_Area.geojson"
POI_SR_path = r"C:/Users/Soheil99/OneDrive - UW/0 Research/UW Tacoma/my copy - Satellite Communities Project/Data/POI Within 300 ft SR/POI_CBG_Right_Outside_PC_SR_Buffer.shp"
POI_CR_path = r"C:/Users/Soheil99/OneDrive - UW/0 Research/UW Tacoma/my copy - Satellite Communities Project/Data/POI Within 300 ft CR/POI_CBG_Right_Outside_PC_CR_Buffer.shp"
save_path = r"C:\Users\Soheil99\OneDrive - UW\0 Research\UW Tacoma\my copy - Satellite Communities Project\Analysis\RuralATGapFinder\data"

In [ ]:
sld_selected_columns = ['GEOID10', 'CSA_Name', 'CBSA_Name', 'Ac_Land',
                        'Ac_Unpr', 'Ac_Water', 'TotPop', 'CountHU',
                        'HH', 'P_WrkAge', 'White', 'Male', 'Residents',
                        'Drivers', 'Vehicles', 'GasPrice', 'Pct_AO0', 
                        'R_LowWageWk', 'R_MedWageWk', 'R_HiWageWk', 'R_PCTLOWWAGE',
                        'E_LowWageWk', 'E_MedWageWk', 'E_HiWageWk', 'E_PctLowWage', 
                        'D3A', 'D3AAO', 'D3AMM', 'D3APO', 'D3B', 'D3BAO', 'D3BMM3',
                        'D3BMM4', 'D3BPO3', 'D3BPO4', 'D4A', 'D4B025', 'D4B050',
                        'D4C', 'D4D', 'D4E', 'D5AR', 'D5AE', 'D5BR', 'D5BE', 'geometry']

In [ ]:
def save_geopackage(gdf, folder_path, filename, driver=None):
    ## Saving the file
    os.makedirs(folder_path, exist_ok=True)  # make sure folder exists
    filepath = os.path.join(folder_path, filename)  # we use .gpkg instead of .shp in order to keep column names
    # gdf.to_file(filepath, driver="GPKG")
    gdf.to_file(filepath, driver=driver)
    

In [ ]:
def geomcollection_to_multipolygon(geom):
    if isinstance(geom, Polygon):
        return MultiPolygon([geom])          # convert single Polygon to MultiPolygon
    elif isinstance(geom, MultiPolygon):
        return geom                          # already fine
    elif isinstance(geom, GeometryCollection):
        # extract only Polygon/MultiPolygon parts
        polygons = [g for g in geom.geoms if isinstance(g, (Polygon, MultiPolygon))]
        if polygons:
            return MultiPolygon([p for poly in polygons for p in (poly.geoms if isinstance(poly, MultiPolygon) else [poly])])
        else:
            return None                       # no polygons inside
    else:
        return None                            # other geometry types

def polygon_to_multipolygon(gdf):
# turn everything into MultiPolygon. 
    gdf["geometry"] = gdf["geometry"].apply(lambda geom: MultiPolygon([geom]) if isinstance(geom, Polygon) else geom)

## Define study area counties

In [371]:
# Get TIGER/Line file for counties in a specific state
wa_counties = pygris.counties(state = state_in, cb=True, year=2023)
studyarea = wa_counties[wa_counties["NAME"].isin(counties_in)]

# Plot study area
fig, ax = plt.subplots(figsize=(8, 8))
studyarea.plot(ax=ax, facecolor="white", edgecolor="gray")
for _, row in studyarea.iterrows():
    plt.annotate(
        row["NAME"], 
        (row.geometry.centroid.x, row.geometry.centroid.y),
        fontsize=8, color="blue", fontweight='bold',
        ha="center",   # horizontal alignment
        va="center"    # vertical alignment
    )
ax.annotate('N', xy=(0.1, 0.95), xytext=(0.1, 0.85),
            arrowprops=dict(facecolor='black', width=3, headwidth=10),
            ha='center', va='center', fontsize=12,
            xycoords='axes fraction')
plt.axis("off")
plt.title(f"{state_in} Study Area Counties")
plt.show()

HTTP download failed, trying FTP as fallback...


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Soheil99\\anaconda3\\envs\\arcgispro-cloned\\Lib\\site-packages\\pygris\\internals\\fips_codes.csv'

In [ ]:
studyarea.explore()

## Load Smart Location Database

In [ ]:
US_SLD_CBG = gpd.read_file(sld_gdb_path, layer="EPA_SLD_Database_V3")

# filter Washington only (STATEFP = 53)
Washington_SLD_CBG = US_SLD_CBG[US_SLD_CBG["STATEFP"] == state_FIPS]

In [ ]:
## different ways to explore data

# Washington_SLD_CBG.head()
# Washington_SLD_CBG.describe()
# Washington_SLD_CBG.info()
# Washington_SLD_CBG.columns.tolist()
# Washington_SLD_CBG.crs
Washington_SLD_CBG['CBSA_Name'].unique()

In [ ]:
### Different ways to show data
Washington_SLD_CBG.plot(figsize=(8, 8), color="white", edgecolor="black")
plt.title("Smart Location Database - WA Block Groups")
plt.show()

In [ ]:
Washington_SLD_CBG.explore(
    column="CSA_Name",   # optional: color counties by a column
    tooltip= ['GEOID20', 'CSA_Name', 'CBSA_Name'],  # show all attributes on hover/click
    popup=True,      # clicking shows attribute table
    cmap="Set3"      # colormap
)

## filter SLD

### filter SLD for study area

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))  # 1 row, 2 columns

# Left subplot
studyarea.plot(ax=ax[0], color="white", edgecolor="black")
ax[0].set_title("Study Area")
ax[0].axis("off")

# Right subplot
Washington_SLD_CBG.plot(ax=ax[1], color="white", edgecolor="black")
ax[1].set_title("CBG")
ax[1].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
# coordinate reference system sync
Washington_SLD_CBG = Washington_SLD_CBG.to_crs(studyarea.crs)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))  # 1 row, 2 columns

# Left subplot
studyarea.plot(ax=ax[0], color="white", edgecolor="black")
ax[0].set_title("Study Area")
ax[0].axis("off")

# Right subplot
Washington_SLD_CBG.plot(ax=ax[1], color="white", edgecolor="black")
ax[1].set_title("CBG")
ax[1].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
#filter CBGs based on county code and land area
study_CBGs = Washington_SLD_CBG[Washington_SLD_CBG['COUNTYFP'].isin(studyarea['COUNTYFP'])]
study_CBGs = study_CBGs[study_CBGs['Ac_Land']>0]

# remove water from land by clipping (NEW** not present in R file) 
WA_SLD_study = gpd.clip(study_CBGs, studyarea)
# we can do this because we had cb=True in pygris.counties(state = state_in, cb=True, year=2023) 


In [ ]:
print(study_CBGs.geometry.type.unique())
print(WA_SLD_study.geometry.type.unique())  # clip function crates some Polygons

In [ ]:
# turn everything into MultiPolygon. 
WA_SLD_study["geometry"] = WA_SLD_study["geometry"].apply(
    lambda geom: MultiPolygon([geom]) if isinstance(geom, Polygon) else geom
)


In [ ]:
# we want to visualize the difference
diff = gpd.overlay(WA_SLD_study, study_CBGs, keep_geom_type=False, how='symmetric_difference')

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(9, 4))

study_CBGs.plot(ax=ax[0])
ax[0].set_title("after filtering")
ax[0].axis("off")

WA_SLD_study.plot(ax=ax[1])
ax[1].set_title("after clip()")
ax[1].axis("off")

diff.plot(ax=ax[2])
ax[2].set_title("symmetric difference")
ax[2].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
diff.explore()

In [ ]:
WA_SLD_study.info()

### keeping necessary attributes

In [ ]:
WA_SLD_study = WA_SLD_study.loc[:, sld_selected_columns]
WA_SLD_study.head()

In [ ]:
WA_SLD_study.explore()

In [ ]:
## Saving the file
save_geopackage(WA_SLD_study, os.path.join(save_path, "WA_AREA_INITIAL"), "WA_AREA_INITIAL.gpkg", driver="GPKG") 
# folder = os.path.join(save_path, "WA_AREA_INITIAL")
# os.makedirs(folder, exist_ok=True)  # make sure folder exists

# filepath = os.path.join(folder, "WA_AREA_INITIAL.gpkg")  # we use .gpkg instead of .shp in order to keep column names
# WA_SLD_study.to_file(filepath, driver="GPKG")


### income Filtering

In [ ]:
# LowWage_Category_Home
median_low_wage_home = WA_SLD_study["R_PCTLOWWAGE"].median(skipna=True) # Percentage of low-wage workers who live in these census tracts.

# Create a new categorical column
WA_SLD_study["LowWage_Category_Home"] = np.where(
    WA_SLD_study["R_PCTLOWWAGE"] > median_low_wage_home,
    "Above Median",
    "Below Median"
)

In [ ]:
WA_SLD_study.explore(
    column="LowWage_Category_Home",         # column to color by
    cmap=["cadetblue", "salmon"],           # colors for categories
    legend=True,                            # show legend
    tooltip=True                             # hover to see attributes
)

In [ ]:
# LowWage_Category_Work
median_low_wage_work = WA_SLD_study["E_PctLowWage"].median(skipna=True) # Percentage of low-wage workers who work in these census tracts.

WA_SLD_study["LowWage_Category_Work"] = np.where(
    WA_SLD_study["E_PctLowWage"] > median_low_wage_work,
    "Above Median",
    "Below Median"
)

In [ ]:
WA_SLD_study.explore(
    column="LowWage_Category_Work",         # column to color by
    cmap=["darkorange", "royalblue"],           # colors for categories
    legend=True,                            # show legend
    tooltip=True                             # hover to see attributes
)

In [ ]:
WA_SLD_study['LowWage_Combined_home_work'] = WA_SLD_study["LowWage_Category_Home"].astype(str) + "_" + WA_SLD_study["LowWage_Category_Work"].astype(str)


WA_SLD_study = WA_SLD_study.to_crs(32610)

save_geopackage(WA_SLD_study, os.path.join(save_path, "WA_AREA_INITIAL"), "WA_AREA_INCOME.gpkg", driver="GPKG") 

In [ ]:
# WA_SLD_study.value_counts()
a = WA_SLD_study['LowWage_Combined_home_work'].value_counts().reset_index()
# locale_counts.columns = ['LOCALE', 'count']

a

## POPULATION CENTERS
This data layer assists WSDOT in prioritizing active transportation improvements in areas where people congregate and access destinations, and where travel distances between destinations align with typical distances travelled by users of pedestrian and bicycle modes. These areas are a priority because they serve the broadest range of users and potential users of the transportation system, including the very young, very old, and people with disabilities.

In [370]:
population_centers = gpd.read_file(pop_ctr_path)
population_centers = population_centers.to_crs(32610)

In [ ]:
m = WA_SLD_study.explore(name='study area CBGs',
                         style_kwds={"weight": 0.2, "opacity": 0.6}, 
                         color='cyan')

population_centers.explore(name='population centers', color='blue',    
                           style_kwds={"weight": 0.5, "opacity": 0.8},  # thinner, lighter border
                           m=m, legend=True)


In [373]:
# # Keep full (uncut) population centers that intersect the CBGs
# population_centers_CBG <- st_filter(
#   population_centers,
#   WA_AREA_SLD_CBG,
#   .predicate = st_intersects
# )
population_centers_study_area = gpd.clip(population_centers, WA_SLD_study)
# population_centers_study_area = gpd.clip(population_centers, studyarea.to_crs(32610))


In [374]:
population_centers_study_area

,OBJECTID,PlaceName,PlaceType,OnHighwayN,ShapeSTAre,ShapeSTLen,geometry
793,794,Vancouver,City/Town,1,1.373992e+09,407007.824460,"MULTIPOLYGON (((518616.205 5059549.418, 518619..."
490,491,Minnehaha CDP,Census Designated Place,1,6.134331e+07,37795.503517,"POLYGON ((530969.02 5057211.841, 530969.378 50..."
313,314,Hazel Dell CDP,Census Designated Place,1,1.341980e+08,62398.942950,"POLYGON ((529550.147 5059954.702, 529550.233 5..."
261,262,Five Corners CDP,Census Designated Place,1,1.607262e+08,64474.535474,"POLYGON ((532445.835 5061659.789, 532552.864 5..."
559,560,Orchards CDP,Census Designated Place,1,1.484787e+08,61588.106592,"POLYGON ((537644.727 5061724.618, 537644.77 50..."
...,...,...,...,...,...,...,...
245,246,Everson UGA,Urban Growth Area,1,1.223127e+07,50829.691897,"MULTIPOLYGON (((550169.591 5418409.664, 550171..."
528,529,Nooksack,City/Town,1,2.560064e+07,33082.340955,"POLYGON ((549960.281 5418613.912, 549775.53 54..."
431,432,Lynden UGA,Urban Growth Area,1,3.092658e+07,47629.830686,"MULTIPOLYGON (((539285.928 5423629.964, 539285..."
430,431,Lynden,City/Town,1,1.534609e+08,82293.809779,"POLYGON ((538419.811 5420197.332, 538407.52 54..."


In [ ]:
population_centers_study_area.explore()

In [ ]:
# Let see the CBGs that do not intersect population centers

# Find intersections
intersects = WA_SLD_study.geometry.apply(
    lambda x: population_centers.intersects(x).any()
)
WA_SLD_study['intersects_w_pop_center'] = intersects

# Keep only those that do NOT intersect
# WA_SLD_study_Outside = WA_SLD_study[~intersects]
WA_SLD_study_Outside = WA_SLD_study[WA_SLD_study.intersects_w_pop_center==False]
print(len(WA_SLD_study_Outside))      

In [ ]:
WA_SLD_study_Outside.explore()

In [ ]:
# I want to keep only CBGs that intercept population centers data.
WA_SLD_study_PC = WA_SLD_study[WA_SLD_study.intersects_w_pop_center==True]
len(WA_SLD_study_PC)
#3590 CBGs, about 96.77% of the initial number of CBGs

In [ ]:
WA_SLD_study_PC.explore()

In [ ]:
## Get the parts of CBGs that are Just *outside* the population centers 
# based on the original example, we should get 1335 rows in the output dataframe.
# --- original R code:
# WA_CBG_outside_PCs <- st_difference(
#   WA_AREA_PC_SLD_CBG,
#   st_union(population_centers)  # Union all population center polygons into one
# )

## 1- One way is to use overlay() function.  -------------------------------------------------------------------------
# diff = gpd.overlay(study_CBGs[WA_SLD_study.intersects_w_pop_center==True].to_crs(32610), population_centers, how='difference')
# diff = gpd.overlay(WA_SLD_study_PC, population_centers, keep_geom_type=True, how='difference')
# diff.explore()  # if we look at the map, there will be super small polygons, on Seattle for example, that are not in the R file and the original example

## 2- The other way is to exactly replicate the R file  --------------------------------------------------------------
pop_union = unary_union(population_centers_study_area.geometry) # Union all population centers into one geometry
WA_CBG_outside_PCs = WA_SLD_study_PC.copy()
# WA_CBG_outside_PCs = study_CBGs[WA_SLD_study.intersects_w_pop_center==True].to_crs(32610) 
# if we use this which is the study CBGs with water, we can get exactly 1335 rows
WA_CBG_outside_PCs["geometry"] = WA_CBG_outside_PCs.geometry.difference(pop_union) # Geometric difference: keep only the "outside" part
#This line removes non polygons from geometrycollection entries
WA_CBG_outside_PCs = WA_CBG_outside_PCs[~WA_CBG_outside_PCs.geometry.is_empty]  # this results in a similar map with 1333 rows.
# if we use: WA_CBG_outside_PCs = study_CBGs[WA_SLD_study.intersects_w_pop_center==True].to_crs(32610) 
# which is the study CBGs with water, we can get exactly 1335 rows

In [ ]:
# if we get geometryCollection entries (which may contain lines) we remove those lines and only keep polygons
mask = WA_CBG_outside_PCs.geometry.type == 'GeometryCollection'
WA_CBG_outside_PCs.loc[mask, "geometry"] = WA_CBG_outside_PCs.loc[mask, "geometry"].apply(geomcollection_to_multipolygon)
WA_CBG_outside_PCs = WA_CBG_outside_PCs[~WA_CBG_outside_PCs.geometry.is_empty]  
# now let's keep everything as MultiPolygons
polygon_to_multipolygon(WA_CBG_outside_PCs)

In [ ]:
len(WA_CBG_outside_PCs)

In [ ]:
WA_CBG_outside_PCs.geometry.type.unique()


In [ ]:
WA_CBG_outside_PCs.explore()

In [ ]:
a = WA_CBG_outside_PCs.copy()
b = population_centers_study_area.copy()
c = WA_SLD_study_Outside.copy()

a["layer"] = "Outside PCs"
b["layer"] = "Population centers"
c["layer"] = "No intersection"

combined = pd.concat([a, b, c], ignore_index=True)

# Optional: fix category order and colors
combined["layer"] = pd.Categorical(combined["layer"],
                                   categories=["Outside PCs", "Population centers", "No intersection"])

# Use a list of colors (one per category)
combined.explore(
    column="layer",
    categorical=True,
    cmap=["cyan", "blue", "gray"],   # colors correspond to the category order above
    legend=True,
    style_kwds={"weight": 0.4, "opacity": 0.7}
)

In [ ]:
print(len(population_centers_study_area)) # Population centers within the initial study area
print(len(WA_CBG_outside_PCs)) # CBGs that intersect population centers (but, just the parts of CBGs adjacent or right outside of population centers)
print(len(WA_SLD_study_Outside)) # CBGs that do not intersect any population centers

In [ ]:
# mapview(WA_CBG_outside_PCs, zcol = "LowWage_Combined", col.regions = c("red4", "red", "brown1", "grey"), legend = TRUE)

# defining the order
# WA_CBG_outside_PCs["LowWage_Combined"] = pd.Categorical( WA_CBG_outside_PCs["LowWage_Combined"],
#                                                          categories=["Above Median_Above Median", "Above Median_Below Median", 
#                                                                      "Below Median_Above Median", "Below Median_Below Median"])
# wage_map = WA_CBG_outside_PCs.explore(column='LowWage_Combined',
#                                       cmap=["gray", "lightcoral", "red", "maroon"], 
#                                       legend=True,
#                                       style_kwds={"weight": 0.3, "opacity": 0.7})

# We can analyze the four groups, but should really focus on the last three, with the last group of CBGs considered as the most affected.


WA_CBG_outside_PCs["LowWage_Combined_home_work"] = pd.Categorical( WA_CBG_outside_PCs["LowWage_Combined_home_work"],
                                                         categories=["Above Median_Above Median", "Above Median_Below Median", 
                                                                     "Below Median_Above Median", "Below Median_Below Median"])
wage_map = WA_CBG_outside_PCs.explore(column='LowWage_Combined_home_work',
                           cmap=["gray", "lightcoral", "red", "maroon"],
                           legend=True)


In [ ]:
wage_map

In [ ]:
wage_pc_map = population_centers_study_area.explore(m=wage_map, style_kwds={"weight": 0.3, "opacity": 0.7})

In [ ]:
wage_pc_map

In [ ]:
WA_SLD_study_Outside.explore(column = 'intersects_w_pop_center',
                             m=wage_pc_map,
                             cmap=['white'],
                             legend=True,
                             style_kwds={"weight": 0.3, "opacity": 0.7})

## NCES DATA


### Read the data


In [ ]:
nces_0 = gpd.read_file(nces_WA_path)
nces_0 = nces_0.to_crs(32610)

In [ ]:
area_type = nces_0.copy()
area_type["LOCALE"] = area_type["LOCALE"].astype(int)

# Define conditions
conditions = [
    area_type["LOCALE"].isin([11, 12, 13]),
    area_type["LOCALE"].isin([21, 22, 23]),
    area_type["LOCALE"].isin([31, 32, 33]),
    area_type["LOCALE"].isin([41, 42, 43])
]

# Define corresponding choices
choices = ["City", "Suburban", "Town", "Rural"]

# Apply np.select
area_type["LOCALE"] = np.select(conditions, choices, default=nces_0["LOCALE"].astype(str))

# Check result
area_type.head()

In [ ]:
m=area_type.explore(column='LOCALE')
m2 = WA_CBG_outside_PCs.explore(m=m)

In [ ]:
m

In [ ]:
m2

### intersect with WA_CBGs_outside_PCs

now we find the area type of each CBG that intersects with population centers

In [ ]:
# Ensure both GeoDataFrames use the same Coordinate Reference System (CRS)
if WA_CBG_outside_PCs.crs != area_type.crs:
    area_type = area_type.to_crs(WA_CBG_outside_PCs.crs)


# Fix any invalid geometries to prevent errors during intersection
WA_CBG_outside_PCs.geometry = WA_CBG_outside_PCs.geometry.make_valid()
area_type.geometry = area_type.geometry.make_valid()
# Use simplify with a tolerance of 0 as an extra step to repair geometries
area_type.geometry = area_type.geometry.simplify(tolerance=0)


# R: sf::sf_use_s2(FALSE)
# Note: This is not needed in Geopandas, which uses a planar geometry engine by default.

# R: intersection <- st_intersection(WA_CBG_outside_PCs, AREA_TYPE)
intersection_gdf = gpd.overlay(WA_CBG_outside_PCs, area_type, how='intersection')
# R: intersection$area <- st_area(intersection)
intersection_gdf['area'] = intersection_gdf.geometry.area

# R: sf::sf_use_s2(TRUE)
# Note: Not applicable to Geopandas.

# R: largest_intersection <- intersection %>%
# R:   group_by(GEOID10) %>%
# R:   summarize(LOCALE = LOCALE[which.max(area)], max_area = max(area))
# R:
# R: largest_intersection_df <- largest_intersection %>%
# R:   st_drop_geometry() %>%
# R:   distinct(GEOID10, .keep_all = TRUE)

# Python equivalent: Find the row with the largest area for each GEOID10
# A common pandas method is to sort and drop duplicates.
intersection_gdf = intersection_gdf.sort_values('area', ascending=False)
largest_intersection_df = intersection_gdf.drop_duplicates(subset='GEOID10', keep='first')
# We only need the key and the column to be merged ('GEOID10', 'LOCALE', and 'area' for the next step)
# This mimics the creation of 'largest_intersection_df' in R.
largest_intersection_df = largest_intersection_df[['GEOID10', 'LOCALE', 'area']].rename(columns={'area': 'max_area'})


# R: WA_CBG_outside_PCs <- WA_CBG_outside_PCs %>%
# R:   left_join(largest_intersection_df, by = "GEOID10")
WA_CBG_outside_PCs = WA_CBG_outside_PCs.merge(
    largest_intersection_df,
    on='GEOID10',
    how='left'
)

# R: WA_CBG_outside_PCs <- subset(WA_CBG_outside_PCs, select=-c(max_area))
WA_CBG_outside_PCs = WA_CBG_outside_PCs.drop(columns=['max_area'])

# R: print(nrow(WA_CBG_outside_PCs))
print(f"Number of rows: {len(WA_CBG_outside_PCs)}")


In [ ]:
WA_CBG_outside_PCs.head()

In [ ]:
# Fixing missing LOCALEs

WA_CBG_outside_PCs.geometry = WA_CBG_outside_PCs.geometry.make_valid()
area_type.geometry = area_type.geometry.make_valid()
area_type.geometry = area_type.geometry.simplify(tolerance=0)

missing_locale_mask = WA_CBG_outside_PCs['LOCALE'].isna()
cbgs_to_fix = WA_CBG_outside_PCs[missing_locale_mask]
print(f'fixing missing LOCALE for GEOID10s:{cbgs_to_fix.GEOID10.to_list()}')

if not cbgs_to_fix.empty:
    # `sjoin_nearest` does all the distance calculations, finds the minimum,
    # and joins the attributes in one efficient step.
    joined = gpd.sjoin_nearest(cbgs_to_fix, area_type, how="left")
    # Update the original GeoDataFrame using the results from the join.
    # The locale from the nearest 'area_type' polygon is in the 'LOCALE_right' column.
    WA_CBG_outside_PCs.loc[missing_locale_mask, 'LOCALE'] = joined['LOCALE_right'].values

missing_locale_mask = WA_CBG_outside_PCs['GEOID10'].isna()
print("\nChecking for any remaining missing locales:")
if not missing_locale_mask.any():
    print("No missing locales found. ✅")
else:
    print(missing_locale_mask)

In [ ]:
locale_counts = WA_CBG_outside_PCs['LOCALE'].value_counts().reset_index()
locale_counts.columns = ['LOCALE', 'count']

locale_counts

In [ ]:
def custom_summary(x):
    return pd.Series({
        "N": x.count(),
        "Q1": x.quantile(0.25),
        "Q3": x.quantile(0.75),
        "Mean": x.mean(),
        "SD": x.std(),
        "Median [Min, Max]": f"{x.median():.2f} [{x.min():.2f}, {x.max():.2f}]"
    })

def categorical_summary(df, col4rows, col4columns):
    counts = (
        df
        .groupby([df[col4rows], df[col4columns]], observed=True)
        .size()
        .unstack(fill_value=0)
    )
    percentages = counts.div(counts.sum(axis=0), axis=1) * 100
    # Combine "count (pct%)"
    combined = counts.astype(str) + " (" + percentages.round(1).astype(str) + "%)"
    return combined  

result = {}
for col in ["R_PCTLOWWAGE", "E_PctLowWage"]:
    summary = (
        WA_CBG_outside_PCs
        .groupby("LowWage_Combined_home_work", observed=True)[col]
        .apply(custom_summary)
        .unstack(level=0)   # groups (Above/Below Median) become columns
    )
    result[col] = summary
    
locale_summary = categorical_summary(WA_CBG_outside_PCs, "LOCALE", "LowWage_Combined_home_work")

# ---------- Combine everything ----------
descript_category_0 = pd.concat({**result, "LOCALE": locale_summary}, axis=0)

descript_category_0

In [ ]:
wa_cbg_outside_pcs_1 = WA_CBG_outside_PCs.copy()
population_centers_filtered_1 = population_centers_study_area.copy()
wa_area_sld_cbg_outside_1 = WA_SLD_study_Outside.copy()

# R: WA_CBG_outside_PCs_1 <- WA_CBG_outside_PCs_1 %>% mutate(...)
# Create a dictionary to recode the values
recode_dict = {
    "Below Median_Below Median": "High Income (Group 1)",
    "Above Median_Above Median": "Low Income (Group 2)",
    "Above Median_Below Median": "Low Income (Group 3)",
    "Below Median_Above Median": "Low Income (Group 4)"
}
wa_cbg_outside_pcs_1['LowWage_Combined_home_work'] = wa_cbg_outside_pcs_1['LowWage_Combined_home_work'].map(recode_dict)

# R: LowWage_Combined = factor(LowWage_Combined, levels = c(...))
# To control the order in the legend, we convert the column to a pandas Categorical type
category_order = [
    "High Income (Group 1)", "Low Income (Group 2)",
    "Low Income (Group 3)", "Low Income (Group 4)"
]
wa_cbg_outside_pcs_1['LowWage_Combined_home_work'] = pd.Categorical(
    wa_cbg_outside_pcs_1['LowWage_Combined_home_work'],
    categories=category_order,
    ordered=True
)

# R: income_colors <- c(...)
# The R named vector translates directly to a Python dictionary
income_colors = {
    "High Income (Group 1)": "deepskyblue",
    "Low Income (Group 2)": "#CD5C5C",  # brown1 is not standard, using indianred
    "Low Income (Group 3)": "red",
    "Low Income (Group 4)":  "#B40426" # red4 is not a standard HTML color, using hex
}

# --- 2. Create the Interactive Map ---

m = wa_cbg_outside_pcs_1.explore(
    column="LowWage_Combined_home_work",
    cmap=[income_colors[cat] for cat in category_order], # Pass a list of colors in the correct order
    categorical=True,
    legend=True, # We'll create the legend and then customize its title
    popup=False,
    name="Satellite Community" # Layer name for the layer control
)


population_centers_filtered_1.explore(
    m=m,
    color="blue",
    style_kwds={'fillOpacity': 0.5, 'weight': 1},
    name="Population Centers"
)
wa_area_sld_cbg_outside_1.explore(
    m=m,
    color="grey",
    style_kwds={'fillColor': 'black', 'fillOpacity': 0.6, 'weight': 1},
    name="Non-Satellite Community"
)

# Add a layer control panel and display the map (same as before)
folium.LayerControl().add_to(m)
m

# saving files

In [ ]:
savepath = 'descript_category_0.xlsx'
descript_category_0.to_excel(savepath)
print(f"saved descriptive summary to {savepath} ")

In [ ]:
WA_CBG_outside_PCs_data_0 = WA_CBG_outside_PCs.drop(columns='geometry')
WA_CBG_outside_PCs_data_0.to_excel("WA_CBG_outside_PCs_data_0.xlsx", index=False)

print("saved data to WA_CBG_outside_PCs_data_0.xlsx")

In [ ]:
columns_to_keep = [
    'GEOID10', 'CSA_Name', 'CBSA_Name', 'R_PCTLOWWAGE', 'E_PctLowWage',
    'LowWage_Category_Home', 'LowWage_Category_Work', 'LowWage_Combined_home_work', 'LOCALE'
]
WA_CBG_outside_PCs_data_1 = WA_CBG_outside_PCs[columns_to_keep]

WA_CBG_outside_PCs_data_1.to_excel("WA_CBG_outside_PCs_data_1.xlsx", index=False)
print("saved subsetted data to WA_CBG_outside_PCs_data_1.xlsx")

### WE NEED NOW TO MAKE SHAPEFILES WE CAN USE IN ARC GIS PRO
for SLD data, geopackage is used since they can hold long column names unlike shapefiles

In [ ]:
save_geopackage(population_centers_study_area, os.path.join(save_path, "POPULATION_CENTERS_WA_AREA"), "POPULATION_CENTERS_WA_AREA.shp", driver=None) 
save_geopackage(WA_SLD_study_Outside, save_path, "WA_CBG_NOT_INTERSECT_PCs.gpkg", driver='GPKG') 
save_geopackage(WA_CBG_outside_PCs, save_path, "WA_CBG_RIGHT_OUTSIDE_PCs.gpkg", driver='GPKG') 

### WE NEED TO MAKE A SHAPEFILE FOR THE POI DATA FOR WA WE CAN USE IN ARC GIS PRO

#### How did Panick get this data? did the website give poi for the specific study area?

In [ ]:
# method1
POI_gdf = gpd.read_file(POI_path)
# this reads the geojson file and skips columns that have bad types (lists)
# now we convert it into shapefile so that arcGisPro can read it
save_geopackage(POI_gdf, os.path.join(save_path, "poi_data"), 'POI_WA_Study_Area.shp')


#method 2  --- open geojson using Json to Features tool -- recommended
# import arcpy
# ans = arcpy.conversion.JSONToFeatures(
#     in_json_file=r"C:\Users\Soheil99\OneDrive - UW\0 Research\UW Tacoma\my copy - Satellite Communities Project\Data\POI Data\WA_Study_Area.geojson",
#     out_features=r"C:\Users\Soheil99\OneDrive - UW\0 Research\UW Tacoma\my copy - Satellite Communities Project\Analysis\RuralATGapFinder\ArcGIS_test\ArcGIS_test.gdb\WA_Study_Area_JSONToFeatures",
#     geometry_type="POINT"
# )

# POI data

### AFTER GETTING THE SHAPEFILE OF POI WITHIN 300 FT OF _ **SR**_ FROM ARCGIS PRO, WE NEED TO FILTER THEM OUT HERE TO KEEP ONLY THOSE THAT COULD BE CONSIDERED AS PRIMARY POI


In [ ]:
POI_Within_SR_Buffer_0 = gpd.read_file(POI_SR_path)

In [ ]:
# Parse the JSON 'categories' Column ---
# Create a function to safely parse the JSON string in each row
def parse_categories(cat_str):
    try:
        return json.loads(cat_str)
    except (json.JSONDecodeError, TypeError):
        # Return a default structure if JSON is malformed or not a string
        return {'primary': None, 'alternate': []}

# Apply the function to create new columns
POI_Within_SR_Buffer_1 = POI_Within_SR_Buffer_0.copy()
POI_Within_SR_Buffer_1['categories_json'] = POI_Within_SR_Buffer_1['categories'].apply(parse_categories)
POI_Within_SR_Buffer_1['primary_category'] = POI_Within_SR_Buffer_1['categories_json'].apply(lambda x: x.get('primary'))
POI_Within_SR_Buffer_1['alternate_categories'] = POI_Within_SR_Buffer_1['categories_json'].apply(lambda x: x.get('alternate', []))

POI_Within_SR_Buffer_1.head()

In [ ]:
# If you want to explore the data using Excel, check the following.

poi_export = POI_Within_SR_Buffer_1.drop(columns='geometry')
# Convert the list of alternate categories to a comma-separated string
poi_export['alternate_categories'] = poi_export['alternate_categories'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
excel_path = "POI_Within_SR_Buffer_1.xlsx"
poi_export.to_excel(excel_path, index=False)
print(f"Saved intermediate data for exploration to Excel at:\n{excel_path}")

In [ ]:
unique_categories = POI_Within_SR_Buffer_1['primary_category'].unique()
print(f"\nFound {len(unique_categories)} unique primary categories. 10 examples:{unique_categories[:10]}")

#### Overture is organized around approximately 22 top-level categories. We need to use the primary categories to filter the data. There are 349 unique primary categories in our shapefile for 1616 POI

In [ ]:
# Filter POIs Based on Primary Category ---

# Define the regex pattern for categories of interest
filter_pattern = r'store|hospital|church|restaurant|salon|food|retailer|shop|post_office|gas_station|park|bar|barber|school|market'
# Use .str.contains() to filter the GeoDataFrame
POI_Within_SR_Buffer_2 = POI_Within_SR_Buffer_1[
    POI_Within_SR_Buffer_1['primary_category'].str.contains(
        filter_pattern,
        case=False,      # ignore_case = TRUE
        na=False,        # Don't match on NaN values
        regex=True
    )
].copy()

print(f"Filtered down to {len(POI_Within_SR_Buffer_2)} relevant POIs.")

In [ ]:
POI_Within_SR_Buffer_2.head()

In [ ]:
#Cleanup for Shapefile Export ---- Probably not needed at all
# Shapefiles do not support list or dictionary columns, so we flatten them.
POI_Within_SR_Buffer_3 = POI_Within_SR_Buffer_2.copy()

# Flatten the 'categories_json' dictionary into a string
POI_Within_SR_Buffer_3['categories_json'] = POI_Within_SR_Buffer_3['categories_json'].apply(
    lambda x: json.dumps(x) # Re-serialize the dictionary to a clean JSON string
)
# Flatten the 'alternate_categories' list into a string
POI_Within_SR_Buffer_3['alternate_categories'] = POI_Within_SR_Buffer_3['alternate_categories'].apply(
    lambda x: ', '.join(map(str, x)) if isinstance(x, list) else ''
)

print("Final data head before writing to shapefile:")
print(f"-- Has {len(POI_Within_SR_Buffer_3)} features.")
POI_Within_SR_Buffer_3.head()

In [ ]:
save_geopackage(POI_Within_SR_Buffer_3, os.path.join(save_path, "POI Within 300 ft SR _ Filtered"), 'POI_Within_SR_Buffer_Filtered.gpkg', driver="GPKG")
# print(f"\nSuccessfully wrote filtered shapefile to:\n{output_path}")


### AFTER GETTING THE SHAPEFILE OF POI WITHIN 300 FT OF _ **CR**_ FROM ARCGIS PRO, WE NEED TO FILTER THEM OUT HERE TO KEEP ONLY THOSE THAT COULD BE CONSIDERED AS PRIMARY POI


In [ ]:
POI_Within_CR_Buffer_0 = gpd.read_file(POI_CR_path)

# Parse the JSON 'categories' Column 
# Apply the function to create new columns
POI_Within_CR_Buffer_1 = POI_Within_CR_Buffer_0.copy()
POI_Within_CR_Buffer_1['categories_json'] = POI_Within_CR_Buffer_1['categories'].apply(parse_categories)
POI_Within_CR_Buffer_1['primary_category'] = POI_Within_CR_Buffer_1['categories_json'].apply(lambda x: x.get('primary'))
POI_Within_CR_Buffer_1['alternate_categories'] = POI_Within_CR_Buffer_1['categories_json'].apply(lambda x: x.get('alternate', []))

POI_Within_CR_Buffer_1.head()

In [ ]:
# If you want to explore the data using Excel, check the following.

poi_export = POI_Within_CR_Buffer_1.drop(columns='geometry')
# Convert the list of alternate categories to a comma-separated string
poi_export['alternate_categories'] = poi_export['alternate_categories'].apply(lambda x: ', '.join(map(str, x)) if isinstance(x, list) else x)
excel_path = "POI_Within_CR_Buffer_1.xlsx"
poi_export.to_excel(excel_path, index=False)
print(f"Saved intermediate data for exploration to Excel at:\n{excel_path}")

In [ ]:
unique_categories = POI_Within_CR_Buffer_1['primary_category'].unique()
print(f"\nFound {len(unique_categories)} unique primary categories. 10 examples:{unique_categories[:10]}")

#### Overture is organized around approximately 22 top-level categories. We need to use the primary categories to filter the data. There are 349 unique primary categories in our shapefile for 1616 POI

In [ ]:
# Filter POIs Based on Primary Category ---

# Define the regex pattern for categories of interest
filter_pattern = r'store|hospital|church|restaurant|salon|food|retailer|shop|post_office|gas_station|park|bar|barber|school|market'
# Use .str.contains() to filter the GeoDataFrame
POI_Within_CR_Buffer_2 = POI_Within_CR_Buffer_1[
    POI_Within_CR_Buffer_1['primary_category'].str.contains(
        filter_pattern,
        case=False,      # ignore_case = TRUE
        na=False,        # Don't match on NaN values
        regex=True
    )
].copy()

print(f"Filtered down to {len(POI_Within_CR_Buffer_2)} relevant POIs.")

In [ ]:
POI_Within_CR_Buffer_2.head()

In [ ]:
#Cleanup for Shapefile Export ---- Probably not needed at all
# Shapefiles do not support list or dictionary columns, so we flatten them.
POI_Within_CR_Buffer_3 = POI_Within_CR_Buffer_2.copy()

# Flatten the 'categories_json' dictionary into a string
POI_Within_CR_Buffer_3['categories_json'] = POI_Within_CR_Buffer_3['categories_json'].apply(
    lambda x: json.dumps(x) # Re-serialize the dictionary to a clean JSON string
)
# Flatten the 'alternate_categories' list into a string
POI_Within_CR_Buffer_3['alternate_categories'] = POI_Within_CR_Buffer_3['alternate_categories'].apply(
    lambda x: ', '.join(map(str, x)) if isinstance(x, list) else ''
)

print("Final data head before writing to shapefile:")
print(f"-- Has {len(POI_Within_CR_Buffer_3)} features.")
POI_Within_CR_Buffer_3.head()

In [ ]:
save_geopackage(POI_Within_CR_Buffer_3, os.path.join(save_path, "POI Within 300 ft CR _ Filtered"), 'POI_Within_CR_Buffer_Filtered.gpkg', driver="GPKG")
# print(f"\nSuccessfully wrote filtered shapefile to:\n{output_path}")
